In [1]:
import itertools
import tempfile
import re

import numpy as np
import pandas as pd
import requests
import scipy.sparse
import tqdm

import xswap
import analysis

%matplotlib inline

## STRING

STRING gives HTTP 403 for `pd.read_table(url)`, but `requests.get` seems to work just fine.

In [2]:
# Download PPI network from STRING
string_url = 'https://stringdb-static.org/download/protein.links.v11.0/9606.protein.links.v11.0.txt.gz'
with tempfile.NamedTemporaryFile() as tf:
    res = requests.get(string_url)
    tf.write(res.content)
    string_df = pd.read_table(tf.name, compression='gzip', sep=' ')

# Download Ensembl to UniProtKB identifier mappings
mapping_url = 'https://string-db.org/mapping_files/uniprot/human.uniprot_2_string.2018.tsv.gz'
with tempfile.NamedTemporaryFile() as tf:
    res = requests.get(mapping_url)
    tf.write(res.content)
    mapping_df = pd.read_table(tf.name, compression='gzip', header=None)

# Create dictionary with mappings
map_to_uniprot = (
    mapping_df
    .assign(uniprot=lambda df: df[1].apply(lambda x: re.search('[A-Z0-9]+', x).group()))
    .set_index(2)
    .loc[:, 'uniprot']
    .to_dict()
)

string_df = (
    string_df
    .assign(
        uniprot_a=lambda df: df['protein1'].map(map_to_uniprot),
        uniprot_b=lambda df: df['protein2'].map(map_to_uniprot),
    )
    .filter(items=['uniprot_a', 'uniprot_b'])
    .dropna()
)
string_nodes = set(string_df.values.flatten())

string_df.head(2)

,uniprot_a,uniprot_b
1,P84085,O43307
2,P84085,O75460


In [3]:
len(string_nodes)

19080

## High-throughput, systematic network

In [4]:
ht_url = 'http://interactome.baderlab.org/data/Raul-Vidal(Nature_2005).psi'
ht_df = pd.read_table(ht_url)

ht_edges_df = (
    ht_df
    .rename(columns={
        'Unique identifier for interactor A': 'ida', 
        'Unique identifier for interactor B': 'idb'})
    .filter(items=['ida', 'idb',])
    .query('ida != "-" and idb != "-"')
    .assign(
        uniprot_a = lambda df: df['ida'].apply(lambda x: re.search('(?<=uniprotkb:)[0-9A-Z]+', x).group()),
        uniprot_b = lambda df: df['idb'].apply(lambda x: re.search('(?<=uniprotkb:)[0-9A-Z]+', x).group())
    )
    .filter(items=['uniprot_a', 'uniprot_b',])
    .drop_duplicates()
)
ht_nodes = set(ht_edges_df.loc[:, 'uniprot_a':'uniprot_b'].values.flatten())

ht_edges_df.head(2)

,uniprot_a,uniprot_b
142,O14964,A0A024R0Y4
144,O95990,A0A024R0Y4


In [5]:
len(ht_nodes)

1510

## Combine data

In [6]:
shared_nodes = sorted(string_nodes.intersection(ht_nodes))
source, target = zip(*itertools.product(shared_nodes, shared_nodes))

len(shared_nodes)

1389

In [7]:
edges_df = (
    pd.DataFrame
    .from_dict({
        'uniprot_a': source,
        'uniprot_b': target,
    })
    .merge(
        string_df
        .assign(string=1),
        on=['uniprot_a', 'uniprot_b'],
        how='left',
    )
    .merge(
        ht_edges_df
        .assign(ht_2014=1),
        on=['uniprot_a', 'uniprot_b'],
        how='left',
    )
    .fillna(0)
)

edges_df.head(2)

,uniprot_a,uniprot_b,string,ht_2014
0,A4D1E9,A4D1E9,0.0,0.0
1,A4D1E9,O00144,0.0,0.0


In [8]:
# Extract edge tuples
string_edges_df = edges_df.query('string == 1')
edges = zip(string_edges_df['uniprot_a'], string_edges_df['uniprot_b'])
edges = list(set(map(tuple, map(sorted, edges))))
mapped_edges, mapping, _ = (
    xswap.preprocessing.map_str_edges(edges, bipartite=False))   

# Create adjacency matrix
sp_mat = analysis.edges_to_matrix(mapped_edges)

In [9]:
# Compute RWR
rwr_mat = analysis.all_pairs_rwr(sp_mat, 0.25)

# Compute RWR p-value
n_perms = 1000
indicator = np.zeros_like(rwr_mat)
perm_edges = mapped_edges.copy()
for i in tqdm.tnrange(n_perms):
    perm_edges, _ = xswap.permute_edge_list(perm_edges, allow_self_loops=True, 
                                            allow_antiparallel=False, seed=i)
    perm_mat = analysis.edges_to_matrix(perm_edges)
    perm_rwr = analysis.all_pairs_rwr(perm_mat, 0.25)
    indicator += (perm_rwr > rwr_mat)
p_values = indicator / n_perms

In [10]:
# Unmap RWR and add to DataFrame
max_id = max(mapping.values())
reversed_map = {v: k for k, v in mapping.items()}
mapped_source, mapped_target = zip(*itertools.product(range(max_id+1), range(max_id+1)))
mapped_rwr_df = (
    pd.DataFrame
    .from_dict({
        'mapped_source': mapped_source,
        'mapped_target': mapped_target,
        'rwr': rwr_mat.flatten(),
        'p_value': p_values.flatten()
    })
    .assign(
        uniprot_a=lambda df: df['mapped_source'].map(reversed_map),
        uniprot_b=lambda df: df['mapped_target'].map(reversed_map)
    )
    .filter(items=['uniprot_a', 'uniprot_b', 'rwr', 'p_value'])
)
edges_df = (
    edges_df
    .merge(mapped_rwr_df, on=['uniprot_a', 'uniprot_b'], how='left')
    .fillna(0)
)

edges_df.to_csv('p_vs_rank.tsv.gz', compression='gzip', sep='\t', index=False)
edges_df.head()

,uniprot_a,uniprot_b,string,ht_2014,rwr,p_value
0,A4D1E9,A4D1E9,0.0,0.0,0.252469,0.094
1,A4D1E9,O00144,0.0,0.0,0.000366,0.212
2,A4D1E9,O00148,1.0,0.0,0.004661,0.000
3,A4D1E9,O00151,1.0,0.0,0.003500,0.002
4,A4D1E9,O00160,0.0,0.0,0.000452,1.000
